## Meta matching v1.0
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.16.4)
* Scipy (1.0.0)
* PyTorch (1.3.1)
* Scikit-learn (0.21.3)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = '/home/the/deepGround/code/2002/Meta_matching_models/'

In [2]:
# initialization and random seed set

import os
import scipy
import torch
import random
import sklearn
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

In [3]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1. load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [4]:
path_v1 = os.path.join(path_repo, 'v1.0')
npz = os.path.join(path_v1, 'meta_matching_v1.0_data.npz')
npz = np.load(npz)
print(npz.files)
x_input = npz['x']
y_input = npz['y']
print(x_input.shape, y_input.shape)

['x', 'y']
(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


### Step 3. Prepare data for PyTorch model
Then we prepare data for DNN model, we will input both the `x_train` and `x_test` into the model to get the predicted phenotypes. 

For meta-matching (DNN) stacking, we do not need real phenotype for the DNN model, I created all zeros `y_dummy` just for function requirement. In some other cases, like meta-matching (DNN) finetuning, you need to use real phenotype data here.

In [6]:
from torch.utils.data import DataLoader
from CBIG_model_pytorch import multi_task_dataset

batch_size = 16
y_dummy = np.zeros(y_train.shape)
dset_train = multi_task_dataset(x_train, y_dummy, True)
trainLoader = DataLoader(dset_train,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=1)

y_dummy = np.zeros(y_test.shape)
dset_test = multi_task_dataset(x_test, y_dummy, True)
testLoader = DataLoader(dset_test,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=1)

### Step 4. load model
Here we load the meta-matching model saved, it is a DNN that takes FC as input and output 67 phenotypes prediction trained on 67 UK Biobank phenotypes

In [7]:
path_model_weight = os.path.join(path_v1, 'meta_matching_v1.0_model.pkl_torch') 
net = torch.load(path_model_weight)
net.to(device)
net.train(False)
print(net)

dnn_4l(
  (fc1): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=87571, out_features=118, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(118, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc2): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=118, out_features=445, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(445, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc3): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=445, out_features=353, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(353, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc4): Sequential(
    (0): Dropout(p=0.28836, inplace=False)
    (1): Linear(in_features=353, out_features=67, bias=True)
  )
)


### Step 5. DNN model predict
Here we apply the DNN trained on 67 UK Biobank phenotypes to predict the 67 phenotypes on `x_train` and `x_test`. We will get the predicted 67 phenotypes on both 80 training subjects and 20 test subjects.

In [8]:
y_train_pred = np.zeros((0, 67))
for (x, _) in trainLoader:
    x= x.to(device)
    outputs = net(x)
    y_train_pred = np.concatenate((y_train_pred, outputs.data.cpu().numpy()), axis=0)
print(y_train_pred.shape, '\n', y_train_pred)

y_test_pred = np.zeros((0, 67))
for (x, _) in testLoader:
    x= x.to(device)
    outputs = net(x)
    y_test_pred = np.concatenate((y_test_pred, outputs.data.cpu().numpy()), axis=0)
print(y_test_pred.shape, '\n', y_test_pred)

(80, 67) 
 [[-0.172272    0.00659015 -0.30208859 ... -0.15929005 -0.38077325
   0.03831939]
 [ 0.08401449 -0.20126963  0.62567258 ...  0.14059752 -0.00804278
   0.46316075]
 [ 0.13499427  0.1402548  -0.51653898 ...  0.0167924  -0.2124238
  -0.15089566]
 ...
 [ 0.14017338 -0.13507895  0.0617892  ...  0.14889172  0.00375667
   0.00102914]
 [-0.00132062 -0.20964415  0.35207626 ...  0.17323519 -0.12568392
  -0.07328422]
 [-0.14659929  0.0975946  -0.24224234 ... -0.07367411 -0.56735682
  -0.06110717]]
(20, 67) 
 [[-0.07091491 -0.20356587  0.30995747 ...  0.17636682 -0.36347133
  -0.133562  ]
 [-0.01349919 -0.13479869  0.26539764 ...  0.22685151  0.03282824
  -0.2556946 ]
 [ 0.038739   -0.22324203  0.34538236 ...  0.21638939 -0.10278898
  -0.06968385]
 ...
 [ 0.01483439 -0.27222618  0.38793996 ...  0.13031593 -0.24093731
   0.35386056]
 [-0.11500867 -0.14910062  0.18525353 ...  0.11365265 -0.3204653
  -0.13270395]
 [ 0.13167757  0.13831383 -0.34880358 ...  0.00669166  0.02046016
  -0.1248268

### Step 6. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model, then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

In [9]:
from CBIG_model_pytorch import stacking

y_test_final, _ = stacking(y_train_pred, y_test_pred, y_train)
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[37.02905892 16.26901877 17.19248669]
 [31.36866311 13.41882507 14.73240677]
 [35.75913507 15.37146875 15.91290481]
 [ 1.14626362  0.4147959   1.54155864]
 [34.79898576 14.69279546 15.74973015]
 [27.34116934 11.72765371 11.77497671]
 [44.9099726  18.98171    20.70634359]
 [15.83518541  6.92195015  7.15008334]
 [20.60853516  8.13286635 10.7118943 ]
 [46.4632692  19.42334593 21.10861999]
 [44.14960856 17.94379747 21.69677942]
 [47.22275847 20.17639219 21.35385446]
 [35.91025987 15.4113871  17.01293692]
 [51.9585441  22.14618668 24.67016   ]
 [51.77015146 21.87992154 24.2454323 ]
 [15.37534639  6.75581305  6.51732385]
 [ 3.89721555  1.63534855  3.28087895]
 [12.23650613  5.16751409  3.92524071]
 [35.39062217 15.35897942 17.24684911]
 [24.0478141   9.71383543 12.31406108]]


### Step 7. Evaluation
Evaluate the prediction performance.

In [10]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_test_final.shape[1]))
for i in range(y_test_final.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.18283692 0.40890475 0.30539004]


### Step 8. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes.

In [11]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov.shape)

(80, 3)
(87571, 3)


### Step 9. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects. The final PNF is in shape of (67, 3), where 67 is number of training phenotypes, and 3 is number of phenotypes.

In [12]:
from CBIG_model_pytorch import covariance_rowwise

cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov.shape)

(67, 3)
